In [ ]:
"""
Assessment 2: Vending Machine
Created on: 10/05/2022
By: Roshanak Ettehadi
"""

# Create a list of dictionary with products including code, name, price and amount.

items = [
    {
        'code' : 1,
        'name' : 'Tea   ',
        'price' : 500,
        'amount' : 10
    },
    {
        'code' : 2,
        'name' : 'Coffee',
        'price' : 800,
        'amount' : 10
    },
    {
        'code' : 3,
        'name' : 'Coke  ',
        'price' : 600,
        'amount' : 10
    },
    {
        'code' : 4,
        'name' : 'Juice ',
        'price' : 900,
        'amount' : 10
    },
]

# Display welcome messages at the beginning.
print("\nWelcome to the Vending Machine!\n")

# Menu that with help fo for loop will go through each item to display them.
def menu(available_products):
    print("Please select from our available products listed below:")
    for item in items:
        # if for one item amount of products is 0, it will not be displayed
        if item['amount'] > 0:
            available_products.append(item['code'])
            print(
                f"Code: {item['code']} - Product Name: {item['name']} - Price in AUD: {item['price']} - Amount: {item['amount']}")


# U-001 New transactions can be started (Users select restart to reset previous transactions and conditions to clear transaction history.)

# It will ask for user input and give the option to see the menu to start the process or quit the program.
def login() :
    while True :
        user = input("Please press Y to see the menu or Q to quit the program: ").upper()
        if user == 'Q':
            cancel()
        if user == 'Y':
            break
        else:
            print("Invalid key. Please input again!")

# Cancel function to quit the program whenever it asked user for their input.
def cancel():
    query = input("If you are sure to quit the machine, enter Q or to continue buying press any key: ").upper()
    if query == 'Q':
        print ("Thank you for using our Vending Machine.\nIt is so sad to see you leaving.\nHope to see you soon!")
        quit()

# Selection process for customers, which will go through the loops to calculate the order's quantity and also decreased the initial amount of the product.
def process_selection(orders, order_number):
    for item in items:
        if item['code'] == order_number:
            orders[item['code']] = orders.get(item['code'], 0) + 1
            item['amount'] -= 1
            if item['amount'] <= 0:
                print("The {} is not available any more!".format(item['name']))
                break
            else :
                print("Invalid key. Please input again!")


# The system shall allow a user to select products, or cancel or processing to the payment.
def select(orders, available_products):
    order = input(f"Enter the code {available_products} or C for Cancel or P for Pay: ").upper()
    return_code = 0
    if order == 'C':
        return_code = 0
    elif order == 'P':
        return_code = 1
    elif order.isnumeric() and int(order) in available_products:
        process_selection(orders, int(order))
        return_code = 2
    else:
        for item in items:
            if item['code'] == int(order):
                print("The {} is not available any more!".format(item['name']))
                break
        return_code = -1
    return return_code


# List of acceptable coins.
aud_coins = [20, 50, 100, 200]

# This function will ask user to enter the amount of money they want to insert and till it reached the order's total cost,
# it will keep asking for money. It also checks if the coin is acceptable.
def insert_coin(totalCredit, total_order_amount, orders) :
    while True :
        print(f"You've inserted ${totalCredit/100:.2f} into the machine so far.")
        coin = input("Please enter the amount of money you'd like to insert or press Q to cancel or R for refunding your money: ").upper()
        if coin == "Q":
            cancel()
            break
        if coin == "R" :
            print(f"Returning ${totalCredit/100:.2f} to you. ")
            totalCredit = 0
            break
        coin = int(coin)
        if coin <= 0 :
            raise ValueError
        if int(coin) in aud_coins :
            totalCredit += coin
            print("Currently there is a total of ${: .2f} in machine".format(totalCredit/100))
            if totalCredit >= total_order_amount :
                print(f"Thank you for buying from us! Please take your", order_name(orders))
                change(totalCredit, total_order_amount, orders)
                coin1 = input(print("Feel free to buy more products or press q to quit the program at any time!")).upper()
                if coin1 == "Q" :
                    cancel()
                    break
            else :
                print("You don't have enough credit in the system to pay for your order!")
                coin2 = input("Please press C for Cancel or any other key to insert more coins: ").upper()
                if coin2 == 'C':
                    reset_canceled_orders(orders)
                else:
                    continue
        else :
            print("The Vending Machine accepts only {}".format(aud_coins), "in cents. \n", end='')
            continue
    return totalCredit

# This function will calculate the remaining money in the system after the purchase.
def change(totalCredit, total_order_amount, orders) :
    remain = totalCredit - total_order_amount
    print("You paid ${: .2f}". format(totalCredit/100), "and you bought with", order_name(orders),
          "You have ${: .2f} remaining in your bank".format(round(remain/100, 2)))

# Write function to write the transaction history into the file.
def print_and_write(file, string) :
    print(string)
    file.write(string + '\n')

# Function to make list of items name in the order
def order_name(orders) :
    order_name = []
    for key, value in orders.items() :
        order_name.append(items[key - 1]['name'])
    return order_name

# Print_order Function to create a bill.
def print_order(orders) :
    file = open('transaction_file.txt', 'w')
    total = 0
    print_and_write(file, "Your orders are:")
    for key, value in orders.items() :
        print_and_write(file, f"{value} - {items[key - 1]['name']}")
        total += value * items[key - 1]['price']
    print_and_write(file, "Total: ${: .2f}". format(total/100))
    file.close()
    return total


# Cancel orders and reset previous transactions and conditions.
def reset_canceled_orders(orders) :
    print("Your order has been cancelled and your previous transaction has been deleted. Please select again: ")
    for key, value in orders.items() :
        items[key - 1]['amount'] += value


# Main Loop which call the login functions and will go through the while loop to check if a order has been submitted and
def main_loop() :
    print("System is working!")
    login()
    orders = {}
    totalCredit = 0
    while True :
        available_products = []
        if len(orders) > 0 :
            total_order_amount = print_order(orders)
        menu(available_products)
        ret = select(orders, available_products)
        if ret == 0 :
            reset_canceled_orders(orders)
            orders = {}
        elif ret == 1 :
            if total_order_amount > 0 :
                totalCredit = insert_coin(totalCredit, total_order_amount, orders)
            else :
                print("No order!")

def main() :
    main_loop()


if __name__ == "__main__" :
   main()

